In [2]:
import cv2
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [3]:
#ahash method 
def aHash(img):
    #Reduce the image scale to 8*8
    img=cv2.resize(img,(8,8),interpolation=cv2.INTER_CUBIC)
    #Grayscale the image (0-255)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #s: sum of pixels value
    s=0
    hash_str=''
    #sum the pixel value
    for i in range(8):
        for j in range(8):
            s=s+gray[i,j]
    #Calculate the average of all the pixels values
    avg=s/64
    #The pixel value greater than the average is marked as 1 and the opposite is 0 for a total of 64 bits.
    for i in range(8):
        for j in range(8):
            if  gray[i,j]>avg:
                hash_str=hash_str+'1'
            else:
                hash_str=hash_str+'0'  
    #Produce hash value (two strings with 1 and 0)
    return hash_str

#dhash method 
def dHash(img):
    img=cv2.resize(img,(9,8),interpolation=cv2.INTER_CUBIC)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    hash_str=''
    #Compare: A pixel value greater than the latter is denoted as 1, and the opposite is denoted as 0. 
    for i in range(8):
        for j in range(8):
            if   gray[i,j]>gray[i,j+1]:
                hash_str=hash_str+'1'
            else:
                hash_str=hash_str+'0'
    return hash_str

#Calculate the Hamming distance and compare
def cmpHash(hash1,hash2):
    n=0
    if len(hash1)!=len(hash2):
        return -1
    for i in range(len(hash1)):
        if hash1[i]!=hash2[i]:
            n=n+1
    return n

#Perceptual hash method 
def pHash(img):
    # resize the DCT (32*32 matrix)
    img = cv2.resize(img, (32, 32))   #interpolation=cv2.INTER_CUBIC
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dct = cv2.dct(np.float32(gray))
    dct_roi = dct[0:8, 0:8]
 
    hash_str=''
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash_str=hash_str+'1'
            else:
                hash_str=hash_str+'0'
    return hash_str

In [4]:
#path = '/Users/liusenyuan/Desktop/input1.jpg'
#img=cv2.imread('/Users/liusenyuan/Desktop/input1.jpg')
#cv2.imshow('image', img)
#image = cv2.imread('/Users/liusenyuan/Desktop/input1.jpg')
#print(image.shape)

## Test run :use the first airplane image as an example, compare in three methods:

In [6]:
img1=cv2.imread('/Users/liusenyuan/Documents/Research/MAT596/Output/sketch_output_RGB/output_data_RGB/airplane_output/airplane1.jpg')
img2=cv2.imread('/Users/liusenyuan/Documents/Research/MAT596/Output/input_data/airplane_input/airplane1.jpg')
hash1= aHash(img1)
hash2= aHash(img2)
print(hash1)
print(hash2)
n1=cmpHash(hash1,hash2)
print('Average Hash value: ',n1)


hash1= dHash(img1)
hash2= dHash(img2)
print(hash1)
print(hash2)
n2=cmpHash(hash1,hash2)
print('Difference Hash value：',n2)

hash1= pHash(img1)
hash2= pHash(img2)
print(hash1)
print(hash2)
n3=cmpHash(hash1,hash2)
print('Perceptual hash value：',n3)
print('The performance value of RGB method: ', (n1+n2+n3)/3)

1111111111111111101111111101010000000000010110010100100000000000
1111111111111111101111111101011000000000010110011100100000000000
Average Hash value:  2
1001101110010110101110001100110111110010000011100110100001010011
1011101110110111101111011100110111010010000011100110100011010011
Difference Hash value： 7
1001001110000100001000000000000000000000001110001000000001000000
1001001110000100001000000000000000000000001110001000000001000000
Perceptual hash value： 0
The performance value of RGB method:  3.0


## Similarity of Airplaine Class

In [7]:
#input_image = None
#output_image = None
hash_value_a = []
hash_value_d = []
hash_value_p = []
for i in range (1,725):
    input_image = cv2.imread(f'/Users/liusenyuan/Documents/Research/MAT596/Output/sketch_output_HSB/output_data_HSB/airplane_output/airplane{i}.jpg')
    output_image = cv2.imread(f'/Users/liusenyuan/Documents/Research/MAT596/Output/input_data/airplane_input/airplane{i}.jpg')
    hash1= aHash(input_image)
    hash2= aHash(output_image)
    n=cmpHash(hash1,hash2)
    hash_value_a.append(n)

for j in range (1,725):
    input_image = cv2.imread(f'/Users/liusenyuan/Documents/Research/MAT596/Output/sketch_output_HSB/output_data_HSB/airplane_output/airplane{j}.jpg')
    output_image = cv2.imread(f'/Users/liusenyuan/Documents/Research/MAT596/Output/input_data/airplane_input/airplane{j}.jpg')
    hash1= dHash(input_image)
    hash2= dHash(output_image)
    n=cmpHash(hash1,hash2)
    hash_value_d.append(n)

for k in range (1,725):
    input_image = cv2.imread(f'/Users/liusenyuan/Documents/Research/MAT596/Output/sketch_output_HSB/output_data_HSB/airplane_output/airplane{k}.jpg')
    output_image = cv2.imread(f'/Users/liusenyuan/Documents/Research/MAT596/Output/input_data/airplane_input/airplane{k}.jpg')
    hash1= pHash(input_image)
    hash2= pHash(output_image)
    n=cmpHash(hash1,hash2)
    hash_value_p.append(n)

average_a = sum(hash_value_a)/len(hash_value_a)
average_d = sum(hash_value_d)/len(hash_value_a)
average_p = sum(hash_value_p)/len(hash_value_p)

print("Compare Airplane Output Class with airplane original images. Similarity: ")
print("In aHash method: ",average_a)
print("In dHash method: ",average_d)
print("In pHash method: ",average_p)
print("The performance of HSB method: ", (average_a+average_d+average_p)/3)

Compare Airplane Output Class with airplane original images. Similarity: 
In aHash method:  5.466850828729282
In dHash method:  12.802486187845304
In pHash method:  2.3729281767955803
The performance of HSB method:  6.8807550644567215


## Cosine Similarity 

In [8]:
def get_thum(image, size=(32, 32), greyscale=False):
    #set image to high quality
    #image = image.resize(size, Image.ANTIALIAS)
    if greyscale:
        # grayscale the image and convert to L
        image = image.convert('L')
    return image

# calculate the cosine distance 
def image_similarity_vectors_via_numpy(image1, image2):
    image1 = get_thum(image1)
    image2 = get_thum(image2)
    images = [image1, image2]
    vectors = []
    norms = []
    for image in images:
        vector = []
        for pixel_tuple in image.getdata():
            vector.append(np.average(pixel_tuple))
        vectors.append(vector)
        norms.append(np.linalg.norm(vector, 2))
    a, b = vectors
    a_norm, b_norm = norms
    # dot product
    res = np.dot(a / a_norm, b / b_norm)
    return res

## Test run

In [9]:
image1 = Image.open(r"/Users/liusenyuan/Documents/Research/MAT596/Output/sketch_output_RGB/output_data_RGB/airplane_output/airplane1.jpg").convert("L") 
image2 = Image.open(r"/Users/liusenyuan/Documents/Research/MAT596/Output/input_data/airplane_input/airplane1.jpg").convert("L") 
image_similarity_vectors_via_numpy(image1,image2)

0.999372916240713

In [ ]:
#for i in range (1,3):
    #path = f'/Users/liusenyuan/Desktop/MAT Independent Study/Output/sketch_output_RGB/output_data_RGB/airplane_output/airplane{i}.jpg'
    #output_image = cv2.imread(f'/Users/liusenyuan/Desktop/MAT Independent Study/Output/input_data/airplane_input/airplane{i}.jpg')
#path

## Test run: compare the first class - airplane

In [10]:
cos_value = []
for i in range (1,50):
    path1 = f'/Users/liusenyuan/Documents/Research/MAT596/Output/sketch_output_RGB/output_data_RGB/airplane_output/airplane{i}.jpg'
    path2 = f'/Users/liusenyuan/Documents/Research/MAT596/Output/input_data/airplane_input/airplane{i}.jpg'
    image1 = Image.open(path1).convert("L") 
    image2 = Image.open(path2).convert("L")
    cos_value.append(image_similarity_vectors_via_numpy(image1,image2))
avg_cos = sum(cos_value)/len(cos_value)
print(avg_cos)

0.9990260956561093


## Plot the relation

In [15]:
H1 = cv2.calcHist([img1], [1], None, [256], [0, 256])
H1 = cv2.normalize(H1, H1, 0, 1, cv2.NORM_MINMAX, -1)  

H2 = cv2.calcHist([img2], [1], None, [256], [0, 256])
H2 = cv2.normalize(H2, H2, 0, 1, cv2.NORM_MINMAX, -1)  

In [17]:
plt.plot(H1,label='input1')
plt.plot(H2,label='output1')
plt.xlim([0, 255])
plt.ylim([0, 1])
plt.title('pixel vs ratio')
plt.show()
#plt.legend()